## 📚 Libraries and Tools I Used

- **OpenCV (`cv2`)**  
  I used OpenCV to handle the webcam feed, process images, and show live video windows. It’s a go-to library for most computer vision tasks.

- **MediaPipe (`mediapipe`)**  
  MediaPipe’s Face Mesh module lets me detect 468 detailed facial landmarks in real-time, which is perfect for measuring facial asymmetry precisely.

- **NumPy (`numpy`)**  
  NumPy helps me with numerical operations like calculating averages and manipulating arrays of landmark coordinates efficiently.

- **Time (`time`)**  
  I used the time module to add countdowns and delays, like waiting a few seconds before taking a snapshot to make sure the face is stable.

---

Using these libraries together, I’m able to capture video, detect and analyze the face accurately, and control the timing of the scan.


In [6]:
import cv2
import mediapipe as mp
import numpy as np
import time

## ⚙️ Setting Up MediaPipe Face Mesh

- **`mp_face_mesh` & `mp_drawing`**  
  I used these shortcuts from MediaPipe to access the face mesh detection and drawing tools. They help me detect facial landmarks and visualize them on the video feed.

- **`drawing_spec`**  
  This defines how the landmarks look when drawn — I chose small circles with thin outlines to keep the visualization clear but not distracting.

- **Initializing `face_mesh`**  
  Here, I set up the Face Mesh detector with some key parameters:  
  - `static_image_mode=False` because I’m working with live video, not still images.  
  - `max_num_faces=1` to focus on detecting only one face at a time (the patient).  
  - `refine_landmarks=True` to get more precise points around important features like eyes and lips.  
  - Detection and tracking confidence thresholds are set to 0.5 to balance accuracy and speed.

- **Symmetric Landmark Pairs**  
  To calculate facial asymmetry, I selected pairs of landmarks on opposite sides of the face that should roughly mirror each other:  
  - Outer and inner eye corners, mouth corners, and cheeks.  
  - Comparing these pairs lets me measure how symmetrical the face is.

*Note:* I only included a few pairs for simplicity, but I plan to add more to improve accuracy later.

---

This setup lets me detect detailed facial landmarks in real time and compare key points to assess asymmetry.


In [7]:
# === MediaPipe setup ===
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# Initialize Face Mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Symmetric landmark pairs for comparison
# TODO: Expand list for better accuracy
symmetric_pairs = [
    (33, 263),   # outer eye corners
    (133, 362),  # inner eye corners
    (61, 291),   # mouth corners
    (199, 429),  # cheek points
]

## 🔍 Calculating Facial Asymmetry

I wrote this function to measure how asymmetrical the face is by comparing pairs of symmetric landmarks.

Here’s how it works:

- For each pair of corresponding points on the left and right sides of the face, I first convert their normalized coordinates to pixel values based on the image width and height.

- Since the right side landmarks need to be compared against the left side, I **mirror** the right points horizontally across the vertical midline of the face.

- Then, I calculate the Euclidean distance between each left landmark and its mirrored right counterpart — basically, how far apart they are in pixels.

- Finally, I average these distances across all pairs to get a single **asymmetry score**: the higher the score, the more asymmetrical the face.

This method gives me a straightforward way to quantify facial asymmetry using precise landmark positions.



In [8]:
def calculate_asymmetry(landmarks, img_width, img_height):
    """
    Calculate average difference between symmetric facial landmarks.
    The right-side landmarks are mirrored across the vertical midline for comparison.
    """
    total_diff = 0
    for left_idx, right_idx in symmetric_pairs:
        left_point = np.array([
            landmarks[left_idx].x * img_width,
            landmarks[left_idx].y * img_height
        ])
        right_point = np.array([
            landmarks[right_idx].x * img_width,
            landmarks[right_idx].y * img_height
        ])
        
        # Mirror the right point horizontally
        mid_x = img_width / 2
        mirrored_right = np.array([2 * mid_x - right_point[0], right_point[1]])
        
        diff = np.linalg.norm(left_point - mirrored_right)
        total_diff += diff
    
    return total_diff / len(symmetric_pairs)


## 🎯 Checking If the Face Is Centered

To make sure the face is properly aligned before taking a snapshot, I wrote this function to check if the face is horizontally centered in the camera frame.

Here’s what I do:

- I take all the detected facial landmarks and calculate their average x-coordinate (left to right position) — this gives me an approximate midpoint of the face.

- Since MediaPipe gives normalized coordinates (from 0 to 1), I convert this average to pixels based on the image width.

- Then, I compare the face midpoint to the center of the image frame.

- If the face midpoint is within a certain threshold (default 10% of the frame width) from the center, I consider the face centered.

This helps me prompt the user to adjust their position for a better scan if needed.


In [10]:
def is_face_centered(landmarks, img_width, threshold=0.1):
    """
    Check if the face is horizontally centered within a threshold.
    - landmarks: list of face landmarks from MediaPipe
    - img_width: width of the image/frame
    - threshold: allowed deviation as fraction of image width (default 10%)
    
    Returns True if centered, False otherwise.
    """
    # Calculate average x of all landmarks (face midpoint)
    x_coords = [lm.x for lm in landmarks]
    avg_x = np.mean(x_coords)  # normalized 0 to 1

    # Convert to pixel coordinate
    face_mid_x = avg_x * img_width
    center_x = img_width / 2

    # Check if within threshold distance from center
    return abs(face_mid_x - center_x) < threshold * img_width

## 📊 Interpreting the Asymmetry Score

After calculating the asymmetry score, I use this function to give a simple interpretation based on clinically inspired thresholds:

- Scores **0–30 px** mean the face is very symmetrical, which is normal.  
- Scores between **30–80 px** indicate mild asymmetry, which is typical for most people.  
- Scores from **80–150 px** suggest noticeable asymmetry that might warrant further attention.  
- Scores above **150 px** signal significant asymmetry, which could be a sign of neurological issues and means the person should see a neurologist immediately.

This helps translate the raw number into something meaningful and actionable for the user.


In [21]:
def interpret_asymmetry_score(score):
    """
    Interpretation of the asymmetry score in pixels.
    Clinical-inspired thresholds:
    0–30 px    : Very symmetrical
    30–80 px   : Mild asymmetry (typical in most human faces)
    80–150 px  : Noticeable asymmetry
    150+ px    : Significant asymmetry
    """
    if score < 30:
        return "Very symmetrical face detected."
    elif score < 80:
        return "Mild asymmetry detected (typical in most human faces)."
    elif score < 150:
        return "Noticeable asymmetry detected."
    else:
        return "Significant asymmetry detected — consult a neurologist immediately."

In [26]:
def show_welcome_screen():
    # Create a blank black image
    height, width = 480, 640
    welcome_img = np.zeros((height, width, 3), dtype=np.uint8)

    # Add welcome text
    lines = [
        "Welcome to the Virtual Neurologist!",
        "",
        "This AI system will check your face for neurological",
        "symptoms by detecting facial asymmetry.",
        "",
        "Instructions:",
        "- Sit comfortably in front of your camera.",
        "- Make sure your face is well lit and visible.",
        "- Follow on-screen prompts to align your face.",
        "",
        "Press any key to begin..."
    ]

    y0, dy = 50, 30
    for i, line in enumerate(lines):
        y = y0 + i * dy
        cv2.putText(welcome_img, line, (30, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Show image and wait for key press
    cv2.imshow("Welcome Screen", welcome_img)
    cv2.waitKey(0)
    cv2.destroyWindow("Welcome Screen")

In [32]:
# === Interactive AI Neurologist Simulation with Alignment Check ===
show_welcome_screen()
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
else:
    print("AI Neurologist: Please align your face with the camera.")

countdown_started = False
countdown_time = 5  # seconds
start_time = None
snapshot = None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)
    img_width = frame.shape[1]

    if results.multi_face_landmarks:
        landmarks = results.multi_face_landmarks[0].landmark
        centered = is_face_centered(landmarks, img_width)

        if centered:
            if not countdown_started:
                start_time = time.time()
                countdown_started = True

            elapsed = time.time() - start_time
            if elapsed < countdown_time:
                instruction = "Hold still, capturing image in..."
                timer_text = f"{int(countdown_time - elapsed)} seconds"
            else:
                snapshot = frame.copy()
                break
        else:
            countdown_started = False
            instruction = "Please center your face in the camera."
            timer_text = ""

    else:
        countdown_started = False
        instruction = "No face detected. Please position your face in front of the camera."
        timer_text = ""

    # Display instructions and timer
    cv2.putText(frame, instruction, (30, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    if timer_text:
        cv2.putText(frame, timer_text, (30, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    # Draw face landmarks if available
    if results.multi_face_landmarks:
        mp_drawing.draw_landmarks(
            image=frame,
            landmark_list=results.multi_face_landmarks[0],
            connections=mp_face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=drawing_spec
        )

    cv2.imshow("Virtual Neurologist", frame)

    if cv2.waitKey(5) & 0xFF in [ord('q'), ord('Q')]:
        break

# Process snapshot if taken
if snapshot is not None:
    frame_rgb = cv2.cvtColor(snapshot, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            mp_drawing.draw_landmarks(
                image=snapshot,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=drawing_spec
            )

            score = calculate_asymmetry(face_landmarks.landmark,
                                        snapshot.shape[1],
                                        snapshot.shape[0])
            
            interpretation = interpret_asymmetry_score(score)

            # Display the score and interpretation clearly
            cv2.putText(snapshot, f"Asymmetry Score: {score:.2f}",
                        (10, 40), cv2.FONT_HERSHEY_SIMPLEX,
                        1, (0, 0, 255), 2)
            cv2.putText(snapshot, interpretation,
                        (10, 80), cv2.FONT_HERSHEY_SIMPLEX,
                        0.8, (255, 255, 255), 2)
            cv2.putText(snapshot, "Press any key to exit.",
                        (10, snapshot.shape[0] - 30), cv2.FONT_HERSHEY_SIMPLEX,
                        0.7, (200, 200, 200), 1)
    else:
        cv2.putText(snapshot, "No face detected. Please retry.",
                    (30, 50), cv2.FONT_HERSHEY_SIMPLEX,
                    0.8, (0, 0, 255), 2)

    cv2.imshow("Facial Asymmetry Result", snapshot)
    cv2.waitKey(0)

cap.release()
cv2.destroyAllWindows()


AI Neurologist: Please align your face with the camera.
